In [ ]:
import pandas as pd
import cantools
from python.psa_checksum import psa_checksum

# config esistente
DBC_FILE = 'dbc/psa_aee2010_r3_fixed2.dbc'
LOG_FILE = 'logs/aa0ad8ba95ff270c|00000017--62c22bb216.csv'

print(f"📂 Carico DBC: {DBC_FILE}")
dbc = cantools.database.load_file(DBC_FILE, strict=False)

print(f"📂 Carico CSV: {LOG_FILE}")
df = pd.read_csv(LOG_FILE)

# normalizza l'addr del CSV in int
# il tuo CSV sembra avere '0x2f6' come stringa
def parse_addr(x):
    if isinstance(x, str) and x.startswith("0x"):
        return int(x, 16)
    return int(x)

df["addr_int"] = df["addr"].apply(parse_addr)

# per report finale
errors = []

print("🚀 Controllo TUTTI i messaggi del DBC che hanno un CHECKSUM...\n")

for msg in dbc.messages:
    # trova i segnali che sembrano checksum
    cs_signals = [s for s in msg.signals if "CHECKSUM" in s.name.upper() or "CRC" in s.name.upper()]
    if not cs_signals:
      continue  # questo messaggio non ci interessa

    msg_id = msg.frame_id
    # prendi tutte le righe del log con questo id (su qualunque bus)
    df_msg = df[df["addr_int"] == msg_id]

    if df_msg.empty:
        # nessun dato reale per questo ID → lo segnaliamo ma andiamo avanti
        print(f"⚠️  ID 0x{msg_id:X} ({msg.name}): nessun frame nel log, salto.")
        continue

    # quali bus hanno visto questo messaggio?
    buses = sorted(df_msg["bus"].unique())
    print(f"\n🧪 ID 0x{msg_id:X} ({msg.name}) — checksum: {[s.name for s in cs_signals]} — bus nel log: {buses}")

    for b in buses:
        df_bus = df_msg[df_msg["bus"] == b].copy()
        if df_bus.empty:
            continue

        bad_rows = 0

        for idx, row in df_bus.iterrows():
            # il CSV deve avere i byte grezzi: adattalo se il nome è diverso
            # qui suppongo ci sia una colonna tipo 'raw_bytes' = "00 11 22 ..."
            raw = row.get("raw_bytes") or row.get("data") or row.get("bytes")
            if raw is None:
                # se il formato è diverso, qui va adattato
                continue

            # normalizza in bytearray
            if isinstance(raw, str):
                # supporta "00 11 22" o "001122"
                parts = raw.strip().split()
                if len(parts) == 1:  # forse "001122..."
                    ba = bytearray.fromhex(raw.strip())
                else:
                    ba = bytearray(int(p, 16) for p in parts)
            else:
                continue  # formato non riconosciuto

            for cs in cs_signals:
                calc = psa_checksum(msg_id, cs, bytearray(ba))  # passiamo una copia
                # estratto dal frame reale
                # posizione nel frame: bit start → byte
                byte_i = cs.start // 8
                in_high = (cs.start % 8) >= 4  # nibble alto o basso
                real_byte = ba[byte_i]
                if in_high:
                    extracted = (real_byte >> 4) & 0xF
                else:
                    extracted = real_byte & 0xF

                ok = (extracted == calc)
                if not ok:
                    bad_rows += 1
                    errors.append({
                        "id": msg_id,
                        "name": msg.name,
                        "bus": b,
                        "time": row.get("time"),
                        "signal": cs.name,
                        "extracted": extracted,
                        "calculated": calc,
                        "raw": raw,
                    })

        if bad_rows == 0:
            print(f"  ✅ bus {b}: tutti i frame ok")
        else:
            print(f"  ❌ bus {b}: {bad_rows} frame con checksum sbagliato")

print("\n================== REPORT ==================")
if not errors:
    print("✅ Nessun checksum sbagliato trovato nei messaggi che hanno un CHECKSUM nel DBC.")
else:
    print(f"❌ Trovati {len(errors)} errori:\n")
    for e in errors[:50]:  # non stampare migliaia
        print(f"ID 0x{e['id']:X} ({e['name']}) bus {e['bus']} time={e['time']}: "
              f"{e['signal']} estr={e['extracted']:X} calc={e['calculated']:X}  raw={e['raw']}")
